# Защита персональных данных клиентов

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

## Загрузка данных

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [3]:
df = pd.read_csv('/datasets/insurance.csv')

In [4]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [6]:
df['Возраст'] = df['Возраст'].astype(int)
df['Зарплата'] = df['Зарплата'].astype(int)

In [7]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.359400,1.194200,0.148000
std,0.500049,8.440807,9900.082063,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [8]:
df.corr()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
Пол,1.000000,0.002074,0.014910,-0.008991,0.010140
Возраст,0.002074,1.000000,-0.019093,-0.006692,0.651030
Зарплата,0.014910,-0.019093,1.000000,-0.030296,-0.014963
Члены семьи,-0.008991,-0.006692,-0.030296,1.000000,-0.036290
Страховые выплаты,0.010140,0.651030,-0.014963,-0.036290,1.000000


## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:**
Качество линейной регрессии не изменится


**Обоснование:** 


$$
PP^{-1} = E
$$


$$
(AB)^{-1} =  B^{-1} A^{-1}
$$


$$
(AB)^T =  B^T A^T
$$

$$
R = XP
$$

P - обратимая матрица, на которую будет умножена основная матрица

$$
a_{1} = Rw
$$


$$
a_{1} = R(R^T R)^{-1} R^T y
$$


$$
a_{1} = XP((XP)^T XP)^{-1} (XP)^T y
$$

$$
a_{1} = XP(P^T X^T XP)^{-1} P^T X^T y
$$

$$
a_{1} = XPP^{-1}(X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$


$$
a_{1} = XE(X^T X)^{-1} X^T y
$$


$$
a_{1} = X(X^T X)^{-1} X^T  y
$$


$$
a_{1} = Xw
$$


$$
a_{1} = a
$$

Значение а не меняется, если умножать матрицу признаков на обратимую матрицу

In [202]:
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

In [203]:
RM = np.random.rand(4,4)
RM

array([[0.14373719, 0.71624685, 0.44955447, 0.41642581],
       [0.95372079, 0.7494472 , 0.10351734, 0.9885686 ],
       [0.72443498, 0.48073037, 0.10235624, 0.70283922],
       [0.96040108, 0.80266675, 0.26084026, 0.43024661]])

In [204]:
RMT = np.linalg.inv(RM)
#матрица обратима
RMT

array([[-0.85392544, -2.73099246,  3.66260108,  1.11831276],
       [ 0.19430929,  6.26963137, -9.48352746,  0.8983661 ],
       [ 1.72891041, -8.95964436, 11.68522215, -0.17564881],
       [ 0.49547365, -0.16860224,  2.43248799, -1.74156188]])

In [194]:
def LR2S(features, target):
    model = LinearRegression()
    model.fit(features, target)
    R2 = r2_score(target, model.predict(features))
    return R2

In [195]:
#r2_score исходной матрицы
LR2S_1 = LR2S(features, target)
LR2S_1

0.42494550308169177

In [193]:
#r2_score для features @ RM
features_RM = features @ RM
display(features_RM)
LR2S(features_RM, target)

,0,1,2,3
0,21859.007410,19775.690953,7271.461991,19666.946974
1,16749.135555,15152.349471,5585.150681,15078.079352
2,9256.396116,8373.742701,3089.782307,8335.066976
3,18375.803652,16624.371123,6100.658436,16524.731380
4,11503.217453,10406.976538,3832.360049,10353.633061
...,...,...,...,...
4995,15733.503836,14233.952458,5232.738599,14154.592469
4996,23091.403981,20890.007189,7672.829352,20769.963913
4997,14939.238148,13515.452999,4962.533955,13436.078535
4998,14411.442788,13038.899133,4789.980975,12963.148982


0.42494550308169965

## Алгоритм преобразования

**Алгоритм**
1. Матрица признаков умножается на 8;
2. К матрице признаков прибавляется 5;
3. Матрица признаков умножается на обратимую (квадратную) матрицу, сгенерированную случайным образом.

**Обоснование**

In [183]:
X = (features * 8 + 5) @ RM
X

,0,1,2,3
0,174877.897153,158215.441822,58179.526082,157344.526028
1,133998.922315,121228.709961,44689.035606,120633.585049
2,74057.006798,66999.855804,24726.088611,66689.486042
3,147012.267088,133004.883176,48813.097644,132206.801279
4,92031.577494,83265.726500,30666.710546,82838.014723
...,...,...,...,...
4995,125873.868564,113881.533858,41869.738947,113245.689985
4996,184737.069718,167129.971710,61390.464974,166168.661541
4997,119519.743057,108133.538187,39708.101797,107497.578515
4998,115297.380180,104321.107259,38327.677958,103714.142095


## Проверка алгоритма

In [184]:
LR2S(X, target)

0.42494550308168033

In [185]:
print('Качество модели до преобразования:', LR2S_1,
      '\nКачество модели после преобразования:', LR2S(X, target))

Качество модели до преобразования: 0.42494550308169177 
Качество модели после преобразования: 0.42494550308168033


## Вывод

Данные клиентов страховой компании защищены. При преобразовании качество моделей машинного обучения не ухудшилось.